# Day 10 
## Part 1
I'm going to create a graph of connections, represented by a dictionary of sets of connected nodes. Use a Point class from earlier years.

In [1]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

    def __add__(self, other):
        return self.__class__(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return self.__class__(self.x - other.x, self.y - other.y)

    def __neg__(self):
        return self.__class__(-self.x, -self.y)

    def __hash__(self):
        return hash((self.x, self.y))

    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x > other.x:
            return False
        else:
            return self.y < other.y

    def __iter__(self):
        yield self.x
        yield self.y

    def __mod__(self, other):
        if isinstance(other, Point):
            return self.__class__(self.x % other.x, self.y % other.y)
        else:
            return self.__class__(self.x % other, self.y % other)
        
    def __mul__(self, multiple):
        return self.__class__(self.x * multiple, self.y * multiple)
    

N = Point(0, 1)
S = Point(0, -1)
W = Point(-1, 0)
E = Point(1, 0)

def parse_data(s):
    directions = {
        "|": [N, S],
        "-": [E, W],
        "L": [N, E],
        "J": [N, W],
        "7": [S, W],
        "F": [S, E]
    }
    g = {}
    
    for y, line in enumerate(reversed(s.strip().splitlines())):
        for x, c in enumerate(line):
            p = Point(x, y)
            if c in directions:
                g[p] = {p + d for d in directions[c]}
            elif c == "S":
                starting_point = p
                
    # Find the positions the starting point is connected to
    g[starting_point] = {loc for loc in g if starting_point in g[loc]}
    
    return starting_point, g
                
test_data = parse_data("""7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ""")

test_data

(Point(x=0, y=2),
 {Point(x=0, y=0): {Point(x=0, y=1), Point(x=1, y=0)},
  Point(x=1, y=0): {Point(x=0, y=0), Point(x=1, y=1)},
  Point(x=3, y=0): {Point(x=3, y=1), Point(x=4, y=0)},
  Point(x=4, y=0): {Point(x=3, y=0), Point(x=4, y=1)},
  Point(x=0, y=1): {Point(x=0, y=0), Point(x=0, y=2)},
  Point(x=1, y=1): {Point(x=1, y=0), Point(x=2, y=1)},
  Point(x=2, y=1): {Point(x=1, y=1), Point(x=3, y=1)},
  Point(x=3, y=1): {Point(x=2, y=1), Point(x=4, y=1)},
  Point(x=4, y=1): {Point(x=3, y=1), Point(x=4, y=2)},
  Point(x=1, y=2): {Point(x=0, y=2), Point(x=1, y=3)},
  Point(x=2, y=2): {Point(x=2, y=3), Point(x=3, y=2)},
  Point(x=3, y=2): {Point(x=3, y=3), Point(x=4, y=2)},
  Point(x=4, y=2): {Point(x=3, y=2), Point(x=4, y=1)},
  Point(x=1, y=3): {Point(x=1, y=2), Point(x=2, y=3)},
  Point(x=2, y=3): {Point(x=1, y=3), Point(x=2, y=4)},
  Point(x=3, y=3): {Point(x=3, y=2), Point(x=3, y=4)},
  Point(x=4, y=3): {Point(x=3, y=3), Point(x=4, y=2)},
  Point(x=0, y=4): {Point(x=-1, y=4), Point(x=0

In [2]:
def part_1(data):
    starting_point, g = data
    current_location = next(iter(g[starting_point]))
    locations_visited = {starting_point, current_location}
    steps = 1
    while (
        next_locations := [
            p  
            for p in g[current_location]
            if p not in locations_visited
        ]):
        current_location = next_locations[0]
        locations_visited.add(current_location)
        steps += 1
    # Final step back to start
    steps += 1
    return steps // 2

assert part_1(test_data) == 8

In [4]:
data = parse_data(open("input").read())

part_1(data)

6733

## Part 2
The ability to squeeze between pipes is a pain, not sure how to do this.

I think what I'll do is double the coordinates so the existing pipes have all even coordinates. This allows for odd numbered coordinates between the pipes. Add extra pipe sections in the coordinates between connected nodes. Then for all coordinates that aren't part of the loop see if there is a clear path to the boundary.

First amend the `part_1` function to return just the nodes in the loop and strip everything else out of the graph.

In [5]:
def strip_nodes(data):
    starting_point, g = data
    current_location = next(iter(g[starting_point]))
    locations_visited = {starting_point, current_location}
    while (
        next_locations := [
            p  
            for p in g[current_location]
            if p not in locations_visited
        ]):
        current_location = next_locations[0]
        locations_visited.add(current_location)
    for k in list(g.keys()):
        if k not in locations_visited:
            del g[k]
    return g

Let's visualise this to make debugging easier.

In [6]:
def print_points(ps):
    lines = []
    min_x = min(p.x for p in ps)
    max_x = max(p.x for p in ps)
    min_y = min(p.y for p in ps)
    max_y = max(p.y for p in ps)
    for y in range(max_y, min_y - 1, -1):
        lines.append(
            "".join(
                "#" if Point(x, y) in ps else "."
                for x in range(min_x, max_x + 1)
            )
        )
    return "\n".join(lines)
            
print(print_points(strip_nodes(test_data)))

..##.
.###.
##.##
#####
##...


Double the coordinates, adding odd corrdinates in between connected nodes. We don't need the connections any more so just return a set of coordinates.

In [7]:
def double_map(g):
    new_map = set()
    for p in g:
        new_p = p * 2
        new_map.add(new_p)
        for nbr in g[p]:
            d = nbr - p
            new_map.add(new_p + d)
    return new_map

print(print_points(double_map(strip_nodes(test_data))))

....###..
....#.#..
..###.#..
..#...#..
###...###
#.......#
#.#######
#.#......
###......


In [38]:
def part_2(data):
    ps = double_map(strip_nodes(data))
    min_x = min(p.x for p in ps)
    max_x = max(p.x for p in ps)
    min_y = min(p.y for p in ps)
    max_y = max(p.y for p in ps)
    
    inner = set()
    outer = set()
    
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            if (p := Point(x, y)) not in (ps | inner | outer):
                q = [p]
                connected = {p}
                hit_boundary = False
                while q and not hit_boundary:
                    loc = q.pop()
                    for d in (N, S, E, W):
                        if (nbr := loc + d) not in (ps | connected):
                            if (
                                nbr.x < min_x 
                                or nbr.x > max_x 
                                or nbr.y < min_y
                                or nbr.y > max_y
                            ):
                                hit_boundary = True
                            else:
                                q.append(nbr)
                                connected.add(nbr)
                if not hit_boundary:
                    inner = inner | connected
                else:
                    outer = outer | connected

    even_ps = [
        p 
        for p in inner 
        if p.x % 2 == 0 and p.y % 2 == 0
    ]
        
    # lines = []
    # min_x = min(p.x for p in ps)
    # max_x = max(p.x for p in ps)
    # min_y = min(p.y for p in ps)
    # max_y = max(p.y for p in ps)
    # for y in range(max_y, min_y - 1, -1):
    #     lines.append(
    #         "".join(
    #             "I" if Point(x, y) in even_ps else
    #             "#" if Point(x, y) in ps else "."
    #             for x in range(min_x, max_x + 1)
    #         )
    #     )
    # print("\n".join(lines))

    return len(even_ps)

test_data_2 = parse_data("""FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L""")

assert part_2(test_data_2) == 10

Wrong. Knocking this on the head for today.

In [36]:
part_2(data)

47596
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
4 25
4 26
4 27
4 28
4 29
4 30
4 31
4 32
4 33
4 34
4 35
4 36
4 37
4 38
4 39
4 40
4 41
4 42
4 43
4 44
4 45
4 46
4 47
4 48
4 49
4 50
4 51
4 52
4 53
4 54
4 55
4 56
4 57
4 58
4 59
4 60
4 61
4 62
4 63
4 64
4 65
4 66
4 67
4 68
4 69
4 70
4 71
4 72
4 73
4 74
4 75
4 76
4 77
4 78
4 79
4 80
4 81
4 82
4 83
4 84
4 85
4 86
4 87
4 88
4 89
4 90
4 91
4 92
4 93
4 94
4 95
4 96
4 97
4 98
4 99
4 100
4 101
4 102
4 103
4 104
4 105
4 106
4 107
4 108
4 109
4 110
4 111
4 112
4 113
4 114
4 115
4 116
4 117
4 118
4 119
4 120
4 121
4 122
4 123
4 124
4 125
4 126
4 127
4 128
4 129
4 130
4 131
4 132
4 133
4 134
4 135
4 136
4 137
4 138
4 139
4 140
4 141
4 142
4 143
4 144
4 145
4 146
4 147
4 148
4 149
4 150
4 151
4 152
4 153
4 154
4 155
4 156
4 157
4 158
4 159
4 160
4 161
4 162
4 163
4 164
4 165
4 166
4 167
4 168
4 169
4 170
4 171
4 172
4 173
4 174
4 175
4 176
4 177
4 178
4 179
4 180
4 181
4 182
4 183


KeyboardInterrupt: 